In [ ]:
from trainer import train_SPIDER
import os, pickle, pandas as pd
import scanpy as sc, numpy as np
import utils, sklearn


In [ ]:
def load_DLPFC():
    data_name = "DLPFC_sc_st_ps.pickle"
    with open("data/DLPFC/"+data_name, "rb") as f:
        loaded = pickle.load(f)
    scadata, adata, pseudo_adata = loaded['sc'],loaded['st'],loaded['ps']
    adata.X = adata.X.toarray()
    # sc.pp.normalize_total(adata,target_sum=1e4)
    # sc.pp.log1p(adata)
    adata.uns['rad_cutoff'] = 239
    adata.uns['num_cluster'] = 7
    coord = pd.read_csv("data/DLPFC/spatial.csv",index_col=0)
    scadata.obsm['spatial'] = coord.loc[scadata.obs_names,:][['X','Y']].values.astype(float)
    adata = adata[~adata.obs['Ground Truth'].isna()]

    sc.pp.normalize_total(adata,target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.normalize_total(pseudo_adata, target_sum=1e4)
    sc.pp.log1p(pseudo_adata)

    n = 4000  
    np.random.seed(3156)  # for reproducibility
    random_indices = np.random.choice(pseudo_adata.n_obs, size=n, replace=False)
    pseudo_adata = pseudo_adata[random_indices, :].copy()
    
    return scadata, adata, pseudo_adata

In [ ]:
scd, std, psd = load_DLPFC()
std.raw = std.copy()

In [ ]:
import importlib
importlib.reload(utils)

In [ ]:
params = {'W_recon': 3.5611961149150226, 'W_mmd': 1.109798809633746, 'W_cell': 1.280970162059623, 'k_cutoff': 9, 'lr': 0.0004790792935102623}
k_cutoff = params["k_cutoff"]
rad_cutoff = std.uns['rad_cutoff']
utils.Cal_knn_expression(std, k_cutoff=k_cutoff)
utils.Cal_knn_expression(psd, k_cutoff=k_cutoff)
utils.Cal_Spatial_Net(std, rad_cutoff=rad_cutoff)

In [ ]:
std,model = train_SPIDER(std, psd, key_added='SPIDER',lr=params["lr"],n_epochs=500,device='cuda:2',random_seed=3156,
                                W_recon=params["W_recon"],W_mmd=params["W_mmd"],W_cell=params["W_cell"])

In [ ]:
sc.pp.normalize_total(std,layer='SPIDER',target_sum=1e4)
sc.pp.log1p(std,layer='SPIDER')
std.obsm['SPIDER_pca'] = sc.tl.pca(std.layers['SPIDER'], n_comps=20, svd_solver="arpack", random_state=3156)
import STAGATE_pyG as STAGATE
STAGATE.mclust_R(std, used_obsm='SPIDER_pca',num_cluster=std.uns['num_cluster'],modelNames="EEI",random_seed=3156)
sc.pl.embedding(std,basis="spatial", color="mclust")